In [1]:
!pip3 install turicreate

  Using cached turicreate-6.4.tar.gz (1.9 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/riya/opt/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/cd/n3q7rkq55tl_cmxkz2gmlg4w0000gn/T/pip-install-b4ekm5fq/turicreate_64ba44f7dbe3435285b5e48be2534807/setup.py'"'"'; __file__='"'"'/private/var/folders/cd/n3q7rkq55tl_cmxkz2gmlg4w0000gn/T/pip-install-b4ekm5fq/turicreate_64ba44f7dbe3435285b5e48be2534807/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/cd/n3q7rkq55tl_cmxkz2gmlg4w0000gn/T/pip-wheel-c1wsy6_e
       cwd: /private/var/folders/cd/n3q7rkq55tl_cmxkz2gmlg4w0000gn/T/pip-install-b4ekm5fq/turicreate_64ba44f7dbe3435285b5e48be2534807/
  Complete outpu

In [2]:
!pip install nltk

In [3]:
!conda install gensim worked

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - worked

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [1]:
import re
import gensim
from gensim import corpora, models, similarities
import nltk
import pandas as pd
import os
import codecs
from gensim.models import Word2Vec
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
corpus = pd.read_csv("labeledEligibilitySample1000000.csv", 
                     sep='\t',
                     error_bad_lines=False,
                     header=None)

In [4]:
len(corpus)

1000000

In [5]:
corpus.iloc[:,0].unique()

array(['__label__0', '__label__1'], dtype=object)

In [6]:
corpus.iloc[:,1].head()

0    study interventions are recombinant CD40-ligan...
1    study interventions are Liposomal doxorubicin ...
2    study interventions are BI 836909 . multiple m...
3    study interventions are Immunoglobulins . recu...
4    study interventions are Paclitaxel . stage ova...
Name: 1, dtype: object

In [7]:
corpus_data= pd.DataFrame(corpus.iloc[:,1])

In [8]:
corpus.shape, type(corpus)

((1000000, 2), pandas.core.frame.DataFrame)

In [9]:
corpus.columns = ['label','text']
corpus.head()

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [10]:
corpus_blob = corpus.groupby('label')['text'].apply(' '.join).reset_index()
corpus_blob

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__1,study interventions are Prednisone . patients ...


In [11]:
inclusion_criteria_0 = corpus_blob['text'][0]
len(inclusion_criteria_0),inclusion_criteria_0[0:500]

(83466072,
 'study interventions are recombinant CD40-ligand . melanoma skin diagnosis and no active cns metastases by ct scan or mri study interventions are Liposomal doxorubicin . colorectal cancer diagnosis and cardiovascular study interventions are BI 836909 . multiple myeloma diagnosis and indwelling central venous cateder or willingness to undergo intra venous central line placement study interventions are Immunoglobulins . recurrent fallopian tube carcinoma diagnosis and patients are allowed to receiv')

In [12]:
inclusion_criteria = corpus['text']

In [13]:
stoplist = set('for a of the and to in is at an must be with are but not no none has have other from as prior or except none see below . study , use " one two three four five six patients before start greater than any allowed by for they since'.split())
texts = [[word for word in inclusion_criteria.split() if word not in stoplist]
         for inclusion_criteria in inclusion_criteria]

In [14]:
len(texts)

1000000

In [15]:
texts[11:20]

[['interventions',
  'Androgens',
  'prostate',
  'cancer',
  'diagnosis',
  'patient',
  'characteristics'],
 ['interventions',
  'Bortezomib',
  'multiple',
  'myeloma',
  'diagnosis',
  'renal',
  'insufficiency',
  's'],
 ['interventions',
  'Prednisone',
  'testicular',
  'lymphoma',
  'diagnosis',
  'bulky',
  'disease',
  'greater_than',
  'equal_than',
  'cm',
  'least',
  'dimension'],
 ['interventions',
  'Razoxane',
  'unspecified',
  'adult',
  'solid',
  'tumor',
  'protocol',
  'specific',
  'diagnosis',
  'platelet',
  'count',
  'greater_than',
  'hundred',
  'zero',
  'mm³'],
 ['interventions',
  'Fludarabine',
  'natural_killer',
  'cell',
  'lymphoblastic',
  'leukemia',
  'lymphoma',
  'diagnosis',
  'bilirubin',
  'more',
  'doc',
  'mg',
  'dl',
  'alanine',
  'aminotransferase',
  'alt',
  'aspartate',
  'aminotransferase',
  'ast',
  'alkaline',
  'phosphatase',
  'alp',
  'more',
  'times',
  'upper',
  'limit',
  'normal',
  'uln'],
 ['interventions',
  'Fluda

In [16]:
len(texts), texts[1:10]

(1000000,
 [['interventions',
   'Liposomal',
   'doxorubicin',
   'colorectal',
   'cancer',
   'diagnosis',
   'cardiovascular'],
  ['interventions',
   'BI',
   '836909',
   'multiple',
   'myeloma',
   'diagnosis',
   'indwelling',
   'central',
   'venous',
   'cateder',
   'willingness',
   'undergo',
   'intra',
   'venous',
   'central',
   'line',
   'placement'],
  ['interventions',
   'Immunoglobulins',
   'recurrent',
   'fallopian',
   'tube',
   'carcinoma',
   'diagnosis',
   'receive',
   'required',
   'receive',
   'additional',
   'cytotoxic',
   'regimens',
   'management',
   'recurrent',
   'persistent',
   'disease',
   'more',
   'non',
   'platinum',
   'non',
   'taxane',
   'regimen'],
  ['interventions',
   'Paclitaxel',
   'stage',
   'ovarian',
   'cancer',
   'diagnosis',
   'recovered',
   'effects',
   'recent',
   'surgery',
   'radiotherapy',
   'therapy'],
  ['interventions',
   'Antibodies,',
   'Monoclonal',
   'recurrent',
   'verrucous',
   'carc

In [17]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /Users/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
type(texts)

list

In [19]:
#f = codecs.open('C:\Python26\text.txt', 'r', 'utf-8-sig')
#text = f.read()
tok_corp = nltk.word_tokenize(str(texts))

In [20]:
len(tok_corp), tok_corp[1:20]

(49525886,
 ['[',
  "'interventions",
  "'",
  ',',
  "'recombinant",
  "'",
  ',',
  "'CD40-ligand",
  "'",
  ',',
  "'melanoma",
  "'",
  ',',
  "'skin",
  "'",
  ',',
  "'diagnosis",
  "'",
  ','])

In [21]:
start = time.time()

ct2vec = Word2Vec(
    texts,
    workers=16,
    vector_size=100,  # Use vector_size instead of size
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 32.126611948013306


In [22]:
word_vector = ct2vec.wv['aml']

# Now you can work with the word vector
print(word_vector)

[-2.1145658   1.3511608   0.66350996  0.4745511  -2.7864952   1.5408239
  0.83188546  3.3434525  -0.52998483  0.5644837  -4.000544   -0.20449512
 -1.8018982  -5.0597157  -0.2141961  -1.3699626   1.3617474  -1.1239318
 -0.7897119  -0.04805114 -1.0879755   0.7907672  -0.45735386  2.5695508
 -1.4550229   3.1493175   1.9716183   2.0201507  -2.678331    2.4539492
 -1.8255905  -1.5249468  -1.1627918  -0.6171662   1.6703498   2.5315893
 -0.58376575 -4.1336613   0.21674336 -0.49712202 -5.835768    2.3444736
 -1.8137488   0.62104946 -0.517315   -4.736142   -2.538733   -1.1758428
  2.2463741  -0.3224473   0.74021333 -0.85321003  2.8930547   0.04246891
 -4.258592   -1.7215632   1.9342728  -0.65754443  1.98768     1.31671
 -0.60912734  2.4075463   3.2649481  -1.5067153   1.6887398   0.6633978
 -2.3813567   0.2519183   3.077182    3.8178139  -0.39845133 -1.1383033
 -2.3794765  -1.3364817   0.41527313 -1.0046449   0.4840322   1.2718186
  0.6959984  -0.52758044 -0.306217   -5.170276   -0.5945125   0.

In [23]:
len(word_vector)

100

In [24]:
similar_words1 = ct2vec.wv.most_similar('aml')

# Now you can print the similar words and their similarity scores
for word, score in similar_words1:
    print(word, score)

mds 0.6986521482467651
cml 0.5964751243591309
blast 0.5658512115478516
crthree 0.5626425743103027
blasts 0.5526569485664368
rcmd 0.5455252528190613
myelodysplasia 0.5383407473564148
mpn 0.5287440419197083
massively_parallel 0.5204973816871643
raeb 0.5201051831245422


In [25]:
similar_words2 = ct2vec.wv.most_similar('Sorafenib')

# Now you can print the similar words and their similarity scores
for word, score in similar_words2:
    print(word, score)

Niacinamide 0.9935101270675659
Sunitinib 0.9206367135047913
Tremelimumab 0.9017098546028137
Bevacizumab 0.8968151211738586
Trebananib 0.8850938677787781
Everolimus 0.8839365243911743
Sirolimus 0.878308117389679
Epothilones 0.8662610054016113
Interferons 0.8635228872299194
Vatalanib 0.8524037003517151


In [26]:
similar_words3 = ct2vec.wv.most_similar('Pembrolizumab')

# Now you can print the similar words and their similarity scores
for word, score in similar_words3:
    print(word, score)

Nivolumab 0.9400351047515869
Vaccines 0.8681163191795349
Sirolimus 0.8669161796569824
Antibodies 0.8664478659629822
Carboplatin 0.8616379499435425
Everolimus 0.8598463535308838
Tremelimumab 0.8593465685844421
Immunoglobulins 0.8563290238380432
Bevacizumab 0.8559635877609253
Azacitidine 0.85268634557724


In [27]:
similar_words4 = ct2vec.wv.most_similar('Enzalutamide')

# Now you can print the similar words and their similarity scores
for word, score in similar_words4:
    print(word, score)

Deslorelin 0.8285701870918274
Methyltestosterone 0.8111528754234314
Goserelin 0.8017089366912842
Hormones 0.7985414862632751
Androgens 0.7953587770462036
Estradiol 0.7932674288749695
Choline 0.7912231087684631
Naïve 0.7768749594688416
Antagonists 0.7762370705604553
Listerine 0.7733756303787231


In [28]:
similar_words5 = ct2vec.wv.most_similar('Estrogen')

# Now you can print the similar words and their similarity scores
for word, score in similar_words5:
    print(word, score)

Receptor 0.9319453835487366
Modulators 0.9109615683555603
Selective 0.8811661005020142
Androgen 0.8058294653892517
Female 0.7650042176246643
Endothelin 0.7489271759986877
Antagonists 0.7431687116622925
Naïve 0.7388229370117188
AR(+) 0.7262250781059265
Esterified 0.7202023267745972


In [29]:
similar_words6 = ct2vec.wv.most_similar('platinum')

# Now you can print the similar words and their similarity scores
for word, score in similar_words6:
    print(word, score)

platin 0.7410343885421753
taxane 0.6402767896652222
cisplatin 0.6063947081565857
doublet 0.5912148356437683
carboplatin 0.5754563212394714
irinotecan 0.5285742282867432
docetaxel 0.5185473561286926
paclitaxel 0.5177183747291565
anthracycline 0.5145747661590576
fludarabine 0.5111420154571533


In [30]:
similar_words7 = ct2vec.wv.most_similar('taxane')

# Now you can print the similar words and their similarity scores
for word, score in similar_words7:
    print(word, score)

docetaxel 0.7576215267181396
doublet 0.7339401841163635
taxanes 0.7146059274673462
gemcitabine 0.70925372838974
anthracycline 0.7074609994888306
anthracenedione 0.6992945075035095
pertuzumab 0.6853770613670349
trastuzumab 0.6800011396408081
cisplatin 0.6699714064598083
ifosfamide 0.6696011424064636


In [31]:
similar_words8 = ct2vec.wv.most_similar('CD20')

# Now you can print the similar words and their similarity scores
for word, score in similar_words8:
    print(word, score)

Adoptive 0.9147209525108337
re-directed 0.9054717421531677
Anti-CD19 0.900149941444397
(Tcon) 0.8967012166976929
Regulatory 0.8934257626533508
Coated 0.8914444446563721
Cevimeline 0.8888653516769409
pbl 0.8887392282485962
triab 0.8874592185020447
MASCT:Multiple 0.8872642517089844


In [32]:
similar_words9 = ct2vec.wv.most_similar('Anti-CD19')

# Now you can print the similar words and their similarity scores
for word, score in similar_words9:
    print(word, score)

GORE 0.9385590553283691
roche 0.936242938041687
Adoptive 0.9292804002761841
FAmily 0.9280421733856201
mesenterial 0.9266427755355835
re-directed 0.9257633686065674
Angel 0.9234329462051392
COMBI 0.9233877062797546
CEntered 0.9193718433380127
DC-Tn-MUC1: 0.9188767671585083


In [33]:
similar_words10 = ct2vec.wv.most_similar('Anti-CD20')

# Now you can print the similar words and their similarity scores
for word, score in similar_words10:
    print(word, score)

Anti-CD19 0.9087074398994446
CD20 0.8810739517211914
Adoptive 0.8754544854164124
DC-Tn-MUC1: 0.8707364201545715
re-directed 0.8688539266586304
Miltenyi 0.853929340839386
recognizing 0.8428816795349121
(CMV)-Specific 0.8380458950996399
(Tcon) 0.8365114331245422
tkr 0.8328790664672852


In [34]:
similar_words11 = ct2vec.wv.most_similar('rituximab')

# Now you can print the similar words and their similarity scores
for word, score in similar_words11:
    print(word, score)

brentuximab_vedotin 0.6268271803855896
chop 0.5913394093513489
lenalidomide 0.5738398432731628
bortezomib 0.5692863464355469
cyclophosphamide 0.5524473786354065
radioimmunotherapy 0.5496864914894104
ibrutinib 0.5472772717475891
fludarabine 0.5393707156181335
carfilzomib 0.5363517999649048
fcr 0.5317872166633606
